#### Imports

In [ ]:
# General import and load data
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

# Training and test spliting
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Preprocessing 
from sklearn.preprocessing import StandardScaler

# Estimators
from sklearn.svm import SVC

# Evaluation
from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

# Optimization
from sklearn.model_selection import GridSearchCV

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)

#### Cargar datos

In [ ]:
data = "/Users/ines/dev/CDAW_reto1/LBBYs/data/processed/data_cleaned.csv"
df = pd.read_csv(data)

# Eliminar filas con NaN en las columnas relevantes
columnas_relevantes = ['City', 'State', 'NoEmp', 'UrbanRural', 'NewExist', 'Accept']
df = df.dropna(subset=columnas_relevantes)

df.head()

,Unnamed: 0,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept
0,0,bd9d6267ec5,1523195006,"P-SCAPE LAND DESIGN, LLC",NORTHFIELD,OH,CITIZENS BANK NATL ASSOC,RI,2005-11-01,2006,...,0,2,0,1,0.0,0.0,2005-12-31,8000.0,0.0,1
1,1,9eebf6d8098,1326365010,The Fresh & Healthy Catering C,CANTON,OH,"FIRSTMERIT BANK, N.A.",OH,2005-06-06,2005,...,1,2,1,1,0.0,0.0,2005-07-31,166000.0,0.0,1
2,2,83806858500,6179584001,AARON MASON & HOWE LLC,SAWYERWOOD,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,2003-03-18,2003,...,4,2,1,2,1.0,0.0,2003-03-31,25000.0,0.0,1
3,3,a21ab9cb3af,8463493009,MID OHIO CAR WASH,COLUMBUS,OH,THE HUNTINGTON NATIONAL BANK,OH,1995-06-28,1995,...,0,0,1,0,0.0,0.0,1996-01-31,220100.0,0.0,1
4,4,883b5e5385e,3382225007,Bake N Brew LLC,Newark,OH,THE HUNTINGTON NATIONAL BANK,OH,2009-04-16,2009,...,0,0,0,1,0.0,0.0,2009-05-31,25000.0,0.0,0


#### Hipótesis 1: Impacto de la ubicación (City, State) en la probabilidad de aprobación

In [14]:
# Seleccionar variables predictoras y la variable objetivo "Accept"
X = df[['City', 'State', 'NoEmp', 'UrbanRural', 'NewExist']] 
y = df['Accept']

# Convertir variables categóricas a dummies (City, State y UrbanRural)
X = pd.get_dummies(X, columns=['City', 'State', 'UrbanRural'], drop_first=True)

# Entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Escalado de las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar modelo SVM (clasificador)
svm_clf = SVC(kernel='rbf', random_state=42)
svm_clf.fit(X_train_scaled, y_train)

# Entrenar modelo SVM (clasificador)
svm_clf = SVC(kernel='rbf', random_state=42)
svm_clf.fit(X_train_scaled, y_train)

# Predicciones y evaluación
y_pred = svm_clf.predict(X_test_scaled)
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(cm)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

ValueError: Input X contains NaN.
SVC does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values